In [35]:

import os
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import glob

# Path to SALICON raw data
#pathToImages = '/home/bat/data/salicon/images'
#pathToMapsTrain = '/home/bat/data/salicon/maps/train'
pathToImgs = '/content/drive/MyDrive/ISP/Orignal_Data_Set/Orignal_Image_Data/*.png'
pathToMaps = '/content/drive/MyDrive/ISP/Orignal_Data_Set/Orignal_Label_Data/*.png'
#pathToImages = '/home/bat/salgan/images_test'
#pathToMaps = '/home/bat/salgan/maps_test'

# Path to processed data
#pathToResizedImagesTrain = '/home/bat/data/salicon/images256x192_train'
#pathToResizedMapsTrain = '/home/bat/data/salicon/maps256x192_train'

pathToResizedImages = '/content/drive/MyDrive/ISP/Preprocessed_Dataset/prep_imgs'
pathToResizedMaps = '/content/drive/MyDrive/ISP/Preprocessed_Dataset/prep_lbls'

#pathToResizedImagesTest = '/home/bat/data/salicon/images256x192_test'

INPUT_SIZE = (256, 192)

'''if not os.path.exists(pathToImgs):
    os.makedirs(pathToImgs)
if not os.path.exists(pathToMaps):
    os.makedirs(pathToMaps)
if not os.path.exists(pathToResizedImages):
    os.makedirs(pathToResizedImages)
if not os.path.exists(pathToResizedMaps):
    os.makedirs(pathToResizedMaps)
#if not os.path.exists(pathToResizedImagesTest):
    #os.makedirs(pathToResizedImagesTest)

#print(list_img_files[5])
'''

'if not os.path.exists(pathToImgs):\n    os.makedirs(pathToImgs)\nif not os.path.exists(pathToMaps):\n    os.makedirs(pathToMaps)\nif not os.path.exists(pathToResizedImages):\n    os.makedirs(pathToResizedImages)\nif not os.path.exists(pathToResizedMaps):\n    os.makedirs(pathToResizedMaps)\n#if not os.path.exists(pathToResizedImagesTest):\n    #os.makedirs(pathToResizedImagesTest)\n\n#print(list_img_files[5])\n'

In [15]:

list_img_files = sorted(glob.glob(pathToImgs))
list_map_files = sorted(glob.glob(pathToMaps))

In [17]:
len(list_img_files)

199

In [19]:
list_map_files[65]

'/content/drive/MyDrive/ISP/Orignal_Data_Set/Orignal_Label_Data/D6ooFFsb.png'

In [20]:
for curr_file in range(25):

    imageResized = cv2.resize(cv2.imread(list_img_files[curr_file]), INPUT_SIZE, interpolation=cv2.INTER_AREA)
    mapResized = cv2.resize(cv2.imread(list_map_files[curr_file]), INPUT_SIZE, interpolation=cv2.INTER_AREA)
        
    cv2.imwrite(pathToResizedImages + str(curr_file) + '.png', imageResized)
    cv2.imwrite(pathToResizedMaps + str(curr_file) + '.png', mapResized)


'''for curr_file in range(181,199):

    imageResized = cv2.resize(cv2.imread(list_img_files[curr_file]), INPUT_SIZE, interpolation=cv2.INTER_AREA)
    mapResized = cv2.resize(cv2.imread(list_map_files[curr_file]), INPUT_SIZE, interpolation=cv2.INTER_AREA)
        
    cv2.imwrite(pathToResizedImagesVal + str(curr_file) + '.png', imageResized)
    cv2.imwrite(pathToResizedMapsVal + str(curr_file) + '.png', mapResized)'''

print('Done resizing images.')
print('Done resizing maps.')

Done resizing images.
Done resizing maps.


In [2]:
import os
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import glob

In [3]:
import torch.nn as nn

def conv2d(in_channels, out_channels, kernel_size = 3, padding = 1):
    return nn.Conv2d(in_channels, out_channels, kernel_size = kernel_size, padding = padding)

def deconv2d(in_channels, out_channels, kernel_size = 3, padding = 1):
    return nn.ConvTranspose2d(in_channels, out_channels, kernel_size = kernel_size, padding = padding)

def relu(inplace = True): # Change to True?
    return nn.ReLU(inplace)

def maxpool2d():
    return nn.MaxPool2d(2)

In [4]:
import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import math
from torch.autograd import Variable


def make_conv_layers(cfg):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [maxpool2d()]
        else:
            conv = conv2d(in_channels, v)
            layers += [conv, relu(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


def make_deconv_layers(cfg):
    layers = []
    in_channels = 512
    for v in cfg:
        if v == 'U':
            layers += [nn.Upsample(scale_factor=2)]
        else:
            deconv = deconv2d(in_channels, v)
            layers += [deconv]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512],
    'D': [512, 512, 512, 'U', 512, 512, 512, 'U', 256, 256, 256, 'U', 128, 128, 'U', 64, 64]
}

def encoder():
    return make_conv_layers(cfg['E'])

def decoder():
    return make_deconv_layers(cfg['D'])

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.encoder = encoder()
        self.decoder = decoder()
        self.mymodules = nn.ModuleList([
            deconv2d(64,1,kernel_size=1, padding = 0),
            nn.Sigmoid()
        ])

    def forward(self,x): #
        #print('Input x', x.size())
        x = self.encoder(x)
        #print('After encoder = ', x.size())
        x = self.decoder(x)
        #print('After decoder = ', x.size())
        x = self.mymodules[0](x)
        x = self.mymodules[1](x)
        #print('Final size = ', x.size())
        return x

#g = Generator()
#x = Variable(torch.rand([17, 3, 192, 256]))
#print('Input :', x.size())
#out = g(x)
#print('Output: ', out.size())

In [5]:
import torch.nn as nn
import torch
import math
from torch.autograd import Variable

#os.environ['CUDA_VISIBLE_DEVICES'] = '2'

class Discriminator(nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()
        self.convs = nn.Sequential( # [-1, 4, 256,192]
                conv2d(4,3,kernel_size=1),
                relu(),
                conv2d(3,32,kernel_size=3), #[-1, 32, 256, 192]
                relu(),
                maxpool2d(),
                conv2d(32,64,kernel_size=3), #[-1, 64, 128, 96]
                relu(),
                conv2d(64,64,kernel_size=3), #[-1, 64, 128, 96]
                relu(),
                maxpool2d(), #[-1,64,64,48]
                conv2d(64,64,kernel_size=3), #[-1,64,64,48]
                relu(),
                conv2d(64,64,kernel_size=3),
                relu(),
                maxpool2d(), #[-1,64,32,24]
        )
        self.mymodules = nn.ModuleList([
            nn.Sequential(nn.Linear(64*32*24, 100), nn.Tanh()),
            nn.Sequential(nn.Linear(100,2), nn.Tanh()),
            nn.Sequential(nn.Linear(2,1), nn.Sigmoid())
        ])
        #self._initialize_weights()

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self.num_flat_features(x))
        x = self.mymodules[0](x)
        x = self.mymodules[1](x)
        x = self.mymodules[2](x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                #print(m.weight.data.shape)
                #print('old conv2d layer!')
                #print(m.weight.data.min())
                #print(m.weight.data.max())
                m.weight.data.normal_(0, math.sqrt(2. / n))
                #print('new conv2d layer!')
                #print(m.weight.data.min())
                #print(m.weight.data.max())
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()
                
#D = Discriminator()
#x = Variable(torch.rand([17, 4, 192, 256]))
#model = Discriminator()
#print('Discriminator input', x.size()) #[-1, 4, 192, 256] because 4 comes from 3 color channel + salience layer.
#out = model(x)
#print('Discriminator out ', out.size()) #[-1, 1]

In [40]:
%matplotlib inline
import glob
import os
import cv2
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
#os.environ["CUDA_VISIBLE_DEVICES"]="1"

#from generator import Generator
#from discriminator import Discriminator
from PIL import Image

def to_variable(x,requires_grad=True):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x,requires_grad)

def show(img): # Display rgb tensor image
    pilTrans = transforms.ToPILImage()
    pilImg = pilTrans(img)
    s = np.array(pilImg)
    plt.figure()
    plt.imshow(s)
    
def show_gray(img): # Display grayscale tensor image
    pilTrans = transforms.ToPILImage()
    pilImg = pilTrans(img)
    s = np.array(pilImg)
    plt.figure()
    plt.imshow(s)
    
def show_img_from_path(imgPath):
    pilImg = Image.open(imgPath)
    s = np.array(pilImg)
    plt.figure()
    plt.imshow(s)
    
def predict(model, img):
    to_tensor = transforms.ToTensor() # Transforms 0-255 numbers to 0 - 1.0.
    im = to_tensor(img)
    inp = to_variable(im.unsqueeze(0), False)
    out = model(inp)
    map_out = out.cpu().data.squeeze(0)
    return map_out


pathToResizedImagesVal = '/content/drive/MyDrive/ISP/Preprocessed_Dataset/prep_imgs/'
pathToResizedMapsVal = '/content/drive/MyDrive/ISP/Preprocessed_Dataset/prep_lbls/'

list_img = [k.split('/')[-1].split('.')[0] for k in glob.glob(os.path.join(pathToResizedImagesVal, '*val*'))]
print(len(list_img))

model = Generator()
pretrained_dict = torch.load('./../generator.pkl')
model.load_state_dict(pretrained_dict)
if torch.cuda.is_available():
    model.cuda()
#print(model)
imgName = list_img[400] + ".png"
img_path = pathToResizedImagesVal + imgName
map_ground_truth = pathToResizedMapsVal + imgName
img = cv2.imread(img_path)
sal_predicted = predict(model,img)
show(img)
show_gray(sal_predicted)
show_img_from_path(map_ground_truth)

to_tensor = transforms.ToTensor() # Transforms 0-255 numbers to 0 - 1.0.
im = to_tensor(img)
inp_d = torch.cat((im,sal_predicted),0)
inp_d.unsqueeze_(0)
print(inp_d.shape)
inp_d = to_variable(inp_d)

model = Discriminator()
pretrained_dict = torch.load('./../discriminator.pkl')
model.load_state_dict(pretrained_dict)
if torch.cuda.is_available():
    model.cuda()
output = model(inp_d)
print(output)

0


FileNotFoundError: ignored

In [ ]:
from numpy import load

data = load('/content/drive/MyDrive/ISP/model/gen_model.npz')
lst = data.files
for item in lst:
    print(item)
    print(data[item])

In [45]:
generator_file = sorted(np.load('/content/drive/MyDrive/ISP/model/gen_model.npz'))

In [6]:
generator_file = np.load('/content/drive/MyDrive/ISP/model/gen_model.npz')

In [7]:
generator_file

In [ ]:
lst = generator_file.files
for item in lst:
    print(item)
    print(generator_file[item])

In [10]:
gen_arr_list = []

In [ ]:
lst = generator_file.files
for item in lst:
    print(item)
    gen_arr_list.append(generator_file[item])

In [58]:
x = generator_file.files[2]

In [15]:
gen_arr = np.asarray(gen_arr_list)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [16]:
gen_arr.shape

(54,)

In [33]:
import keras

In [30]:
import tensorflow as tf

In [34]:
gennerator = keras.models.load_model('/content/drive/MyDrive/ISP/model/gen_model.npz')

OSError: ignored